In [1]:
from models import *
from utils.utils import *
from utils.datasets import *
from utils.kittiloader import *


import os
import sys
import time
import datetime
import argparse
import numpy as np

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import NullLocator
import time

from mpl_toolkits.mplot3d import Axes3D


def drawBEV(point_cloud_,color=(0,1,0)):
    ''' Draw a bird eye view using matplotlib
    
    
    '''
    f = plt.figure(figsize=(18, 14))
    ax = f.add_subplot(111)
    ax.scatter(point_cloud_[:,1],point_cloud_[:,0],
                s=0.1, c=[color]*len(point_cloud_))
    ax.set_title('BEV before frustum extraction')
    ax.set_ylim(0,15)
    ax.set_xlim(-10,10)
    ax.set_facecolor((0,0,0))
# #     plt.show()
    
def draw3D(point_cloud_,color=(0,1,0),angle=(60,180)):
    f = plt.figure(figsize=(15, 10))
    ax = f.add_subplot(111, projection='3d')
    ax.set_facecolor((0, 0, 0))
    ax.set_xlim3d(-10,20)
    ax.set_ylim3d(-10,10)
    ax.set_zlim3d(-3,5)
    ax.view_init(angle[0],angle[1])
    ax.grid(False)
    plt.axis('off')
    ax.scatter(point_cloud_[:,0],point_cloud_[:,1],point_cloud_[:,2], s=0.1, c=[(0,1,0)]*len(point_cloud_))
#     plt.show()

axes_limits = [
    [0, 10], # X axis range
    [-10, 10], # Y axis range
    [-3, 10]   # Z axis range
]
axes_str = ['X', 'Y', 'Z']


image_folder = 'examples/'
config_path = 'config/v390.cfg'
weights_path = 'weights/v390_final.weights'
kitti_path = '/home/project/ZijieMA/KITTI/'
class_path = 'data/coco.names'
conf_thres = 0.5
nms_thres = 0.4
batch_size = 1
n_cpu = 16
img_size = 416
use_cuda = True
CUDA_available = torch.cuda.is_available() and use_cuda

detections = torch.tensor([
    [  5.0433, 208.0689, 133.5634, 274.0513,   0.9992,   0.9995,   2.0000],
         [202.3678, 203.0705, 242.1279, 233.4327,   0.9980,   0.9999,   2.0000],
         [118.8502, 203.6419, 208.2199, 263.8680,   0.9891,   0.9758,   2.0000]
])
if detections[0] is not None:
        detections_with_distance = torch.zeros((detections.shape[0],detections.shape[1]+1))
        detections_with_distance[:,:-1] = detections
        
img_id = 8
img_path = 'examples/000008.png'
img_size_after_resize = img_size
lidar_path = '%straining/velodyne/%06d.bin' % (kitti_path, img_id)
calib = calibread('%straining/calib/%06d.txt' % (kitti_path, img_id))
img = cv2.imread('/home/project/ZijieMA/PyTorch-YOLOv3/examples/%06d.png' % img_id, cv2.IMREAD_UNCHANGED)
# img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
img_width_orig = img.shape[1]

img_height_orig = img.shape[0]

pad_x = max(img_height_orig - img_width_orig, 0) * (img_size_after_resize / max(img_width_orig, img_height_orig))
pad_y = max(img_width_orig - img_height_orig, 0) * (img_size_after_resize / max(img_width_orig, img_height_orig))
point_cloud = np.fromfile(lidar_path, dtype=np.float32).reshape(-1, 4)

# Image height and width after padding is removed
unpad_h = img_size_after_resize - pad_y
unpad_w = img_size_after_resize - pad_x
# detections with shape: (x1, y1, x2, y2, object_conf, class_score, class_pred)

P2 = calib["P2"] # 3x4 matris projection matrix after rectification
# （u,v,1） = dot(P2, (x,y,z,1))
Height_of_camera = 1.65
fu = P2[0][0]  # for horizontal position
fv = P2[1][1]  


In [2]:
for detection in detections_with_distance:
    detection = detection.numpy()
    point_cloud = np.fromfile(lidar_path, dtype=np.float32).reshape(-1, 4)

    # detections with shape: (x1, y1, x2, y2, object_conf, class_score, class_pred)
    box_h = ((detection[3] - detection[1]) / unpad_h) * img_height_orig
    box_w = ((detection[2] - detection[0]) / unpad_w) * img_width_orig
    v_upper = ((detection[1] - pad_y // 2) / unpad_h) * img_height_orig
    u_left = ((detection[0] - pad_x // 2) / unpad_w) * img_width_orig
    v_bottom = v_upper + box_h
    u_right = u_left + box_w
    D_rough = Height_of_camera * fv / (v_bottom - img_height_orig/2)
    if D_rough > 0:
        # remove points that are located behind the camera:
        point_cloud = point_cloud[point_cloud[:, 0] > (D_rough - 3), :]
        # remove points that are located too far away from the camera:
        point_cloud = point_cloud[point_cloud[:, 0] < min(80, D_rough + 3), :]
        point_cloud = point_cloud[point_cloud[:,2] > -1.5,:]
        point_cloud = point_cloud[point_cloud[:,2] < -1,:]
        
        ########################################################################
        # point_cloud               n x 4   original xyzr value before cali in velo coordinate
        # point_cloud_xyz           n x 3   xyz value before cali in velo coordinate
        # point_cloud_xyz_hom       n x 4   xyz1 in velo coordinate
        # point_cloud_xyz_camera    n x 4   xyz1 in camera coordinate
        # point_cloud_camera        n x 4   xyzr in camera coordinate
        # img_points_hom            n x 3   uv_
        # img_points                n x 2   UV
        ########################################################################
        
        R0_rect = np.eye(4)
        R0_rect[0:3, 0:3] = calib["R0_rect"] # 3x3 -> 4x4 up left corner
        Tr_velo_to_cam = np.eye(4)
        Tr_velo_to_cam[0:3, :] = calib["Tr_velo_to_cam"] # 3x4 -> 4x4 up left corner

        # point_cloud_xyz = point_cloud[:, 0:3] # num_point x 3 (x,y,z,reflectance) reflectance don't need
        point_cloud_xyz_hom = np.ones((point_cloud.shape[0], 4))
        point_cloud_xyz_hom[:, 0:3] = point_cloud[:, 0:3] # (point_cloud_xyz_hom has shape (num_points, 4))
        # the 4th column are all 1

        # project the points onto the image plane (homogeneous coords):
        # (U,V,_) = P2 * R0_rect * Tr_velo_to_cam * point_cloud_xyz_hom
        # normalize: (U,V,1)
        img_points_hom = np.dot(P2, np.dot(R0_rect, np.dot(Tr_velo_to_cam, point_cloud_xyz_hom.T))).T # (point_cloud_xyz_hom.T has shape (4, num_points))
        img_points = np.zeros((img_points_hom.shape[0], 2))
        img_points[:, 0] = img_points_hom[:, 0]/img_points_hom[:, 2]
        img_points[:, 1] = img_points_hom[:, 1]/img_points_hom[:, 2]

        row_mask = np.logical_and(
                        np.logical_and(img_points[:, 0] >= u_left,
                                       img_points[:, 0] <= u_right),
                        np.logical_and(img_points[:, 1] >= v_upper,
                                       img_points[:, 1] <= v_bottom))

In [4]:
point_cloud_mask = point_cloud[row_mask,:]

In [6]:
# def ransac_with_bbox(point_cloud_with_mask,)
type(point_cloud_mask)

numpy.ndarray

In [ ]:
def ransac_with_bbox(point_cloud_with_mask,n_sample,category):
    """running ransac to return edges for objects

    Parameters:
    point_cloud_with_mask (numpy.ndarray): Description of arg1

    Returns:
    int:Returning value

   """
    if category = 2 or 5 or 7:
    elif category = 0:
    elif category = 1 or 3:
